In [21]:
import pandas as pd
import numpy as np
import re
import copy 
from urllib.request import urlopen, Request
import urllib
import os
import time

In [2]:
ls

 Volume in drive C is OS
 Volume Serial Number is 5285-348E

 Directory of C:\Users\hamish\Documents\art_gan

07/10/2019  07:35 PM    <DIR>          .
07/10/2019  07:35 PM    <DIR>          ..
07/10/2019  07:33 PM    <DIR>          .ipynb_checkpoints
07/09/2019  10:22 PM    <DIR>          art_fullsize
07/10/2019  07:35 PM    <DIR>          art_thumbs
07/09/2019  05:34 PM        57,515,597 Artworks.csv
07/10/2019  07:35 PM            11,887 exploritory.ipynb
07/09/2019  05:47 PM           230,942 fullsize.jpg
07/10/2019  07:33 PM                72 met exploritory.ipynb
07/09/2019  11:24 PM       256,749,062 MetObjects.csv
07/09/2019  05:41 PM             6,831 thumb.jpg
               6 File(s)    314,514,391 bytes
               5 Dir(s)  14,668,713,984 bytes free


In [29]:
df = pd.read_csv("MetObjects.csv")
print(len(df))
departments = ['Drawings and Prints', 'Photographs', 'European Paintings', 'Modern and Contemporary Art']
df = df[df['Classification'].isin(departments)]
print(len(df))
print(sum(df['Object Date'].isnull()))

495900
29358
77


In [18]:
df.columns

Index(['Object Number', 'Is Highlight', 'Is Public Domain', 'Object ID',
       'Department', 'Object Name', 'Title', 'Culture', 'Period', 'Dynasty',
       'Reign', 'Portfolio', 'Artist Role', 'Artist Prefix',
       'Artist Display Name', 'Artist Display Bio', 'Artist Suffix',
       'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date',
       'Artist End Date', 'Object Date', 'Object Begin Date',
       'Object End Date', 'Medium', 'Dimensions', 'Credit Line',
       'Geography Type', 'City', 'State', 'County', 'Country', 'Region',
       'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification',
       'Rights and Reproduction', 'Link Resource', 'Metadata Date',
       'Repository', 'Tags'],
      dtype='object')

In [16]:
print(len(df[df['Department']=='Modern and Contemporary Art']))
print(len(df[df['Department']=='European Paintings']))
print(len(df[df['Department']=='Photographs']))
print(len(df[df['Department']=='Drawings and Prints']))

14791
2935
39302
179547


In [31]:
def convert_date_to_int(date):
    
    try: 
        return int(date)
    except:
        pass
    
    indate = copy.deepcopy(date)
    if pd.isnull(date):
        return np.nan
    
    split_dates = date.split("–")
    split_dates.extend(date.split("-"))
    dates_lst = []
    for dates in split_dates:
        dates_lst.extend(re.findall('\d{4}', date))
      
    try: 
        return int(dates_lst[0])
    except:
        return np.nan
    
df['Object Date'] = df['Object Date'].apply(lambda x: convert_date_to_int(x))

In [32]:
print(len(df))
df = df[df['Object Date'] >= 1850]
df = df[df['Object Date'] <= 2000]
len(df)

29358


27702

In [94]:
import six
from user_agent import generate_user_agent

def download_image(data_dir, objectid, artist_name, fail_counter):
#     try:
        if not isinstance(artist_name, six.string_types):
            print(type(artist_name))
            if np.isnan(artist_name):
                artist_name = "unknown"
        user_agent = generate_user_agent()
        url = ('http://www.metmuseum.org/art/collection/search/' + objectid.strip())
        res = urlopen(Request(url, headers={'User-Agent': user_agent }))
        html = res.read().decode('utf-8')
        
        print(url)
        print(html)
        offset = html.find('"<img id="artwork__image" class="artwork__image" src="')

        if (offset == -1):
            print("offset failed")
            return None
        end = html[offset:].find('" alt="" itemprop="contentUrl">') + offset

        if (end - offset > 300):
            image_names.append(None)
            print("URL Error")
            return

        image_link = html[offset:end]
        print(image_link)
        image_name = artist_name + "_" + str(objectid) + ".jpg"
        image_path = os.path.join(out_dir, image_name)

        image_path = os.path.join(data_dir, image_name)
        print(image_link)
        if not os.path.exists(image_path):
            time.sleep(np.random.random_sample() * 5)
            try:
                image_file = urllib.request.urlretrieve(image_link, image_path)
            except:
                print(image_link)
#     except:
        
#         print("failed to download image")
#         fail_counter += 1
        

In [96]:
fail_counter = 0
for i in range(1):
    download_image("met_images", str(df['Object ID'].iloc[i]), df['Artist Display Name'].iloc[i], fail_counter)
# print(fail_counter)

<class 'float'>
http://www.metmuseum.org/art/collection/search/14988
<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"></head><body style="margin:0px;height:100%"><iframe src="/_Incapsula_Resource?SWUDNSAI=9&xinfo=14-30961498-0%200NNN%20RT%281562808049169%200%29%20q%280%20-1%20-1%20-1%29%20r%280%20-1%29%20B12%284%2c316%2c0%29%20U18&incident_id=1169000330053873222-172615242508993054&edet=12&cinfo=04000000" frameborder=0 width="100%" height="100%" marginheight="0px" marginwidth="0px">Request unsuccessful. Incapsula incident ID: 1169000330053873222-172615242508993054</iframe></body></html>
offset failed
